In [3]:
import streamlit as st
import easyocr
import mysql.connector
import numpy as np
from PIL import Image
import io

def main():
    st.title("Business Card OCR Application")
    uploaded_image = st.file_uploader("Upload an image of the business card", type=["jpg", "jpeg", "png"])
    if uploaded_image is not None:
        st.image(uploaded_image, caption="Uploaded Image", use_column_width=True)
        if st.button("Extract Information"):
            extracted_info = extract_information(uploaded_image)
            display_extracted_info(extracted_info)
            save_to_database(extracted_info, uploaded_image)

def extract_information(uploaded_image):
    reader = easyocr.Reader(['en'])  # Initialize the OCR reader for English
    image = Image.open(uploaded_image)
    image_np = np.array(image)
    result = reader.readtext(image_np)
    extracted_info = {}
    st.write(extracted_info)

    # Process the OCR result and extract relevant information
    for detection in result:
        text = detection[1]  # Extract the text from the OCR detection
        # Custom logic to identify relevant fields
        if 'company' in text.lower():  # Check if the text contains the word 'company'
            # Extract the company name from the text starting from the word 'company'
            start_idx = text.lower().index('company')
            company_name = text[start_idx + len('company'):].strip()
            extracted_info['company_name'] = company_name
        elif 'name' in text.lower():  # Check if the text contains the word 'name'
            extracted_info['card_holder_name'] = text
        elif 'designation' in text.lower():  # Check if the text contains the word 'designation'
            extracted_info['designation'] = text
        elif 'mobile' in text.lower() and 'number' in text.lower():  # Check for mobile number
            extracted_info['mobile_number'] = text
        elif '@' in text:  # Check if the text contains '@' for email
            extracted_info['email_address'] = text
        elif 'http' in text.lower() or 'www' in text.lower():  # Check for website URL
            extracted_info['website_url'] = text
        elif 'area' in text.lower():  # Check if the text contains the word 'area'
            extracted_info['area'] = text
        elif 'city' in text.lower():  # Check if the text contains the word 'city'
            extracted_info['city'] = text
        elif 'state' in text.lower():  # Check if the text contains the word 'state'
            extracted_info['state'] = text
        elif 'pin' in text.lower() and 'code' in text.lower():  # Check for pin code
            extracted_info['pin_code'] = text

    return extracted_info


def display_extracted_info(extracted_info):
    st.subheader("Extracted Information")
    # Display the extracted information using Streamlit components, e.g., st.write, st.table, etc.

def create_connection():
    # Create a connection to the MySQL database
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="Gobi@7819",
        database="card_data"
    )
    return connection

def create_table(connection):
    # Create a table to store business card information if it doesn't exist
    cursor = connection.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS business_cards (
            id INT AUTO_INCREMENT PRIMARY KEY,
            company_name TEXT,
            card_holder_name TEXT,
            designation TEXT,
            mobile_number TEXT,
            email_address TEXT,
            website_url TEXT,
            area TEXT,
            city TEXT,
            state TEXT,
            pin_code TEXT,
            image BLOB
        )
    ''')
    connection.commit()


# ... (previous code)

def save_to_database(extracted_info, uploaded_image):
    # Convert the uploaded image to bytes
    image_bytes = uploaded_image.read()

    # Save the extracted information and the uploaded image to the database
    connection = create_connection()
    create_table(connection)
    cursor = connection.cursor()

    # Extract fields from extracted_info using .get() method with default value ' '
    company_name = extracted_info.get('company_name', ' ')
    card_holder_name = extracted_info.get('card_holder_name', ' ')
    designation = extracted_info.get('designation', ' ')
    mobile_number = extracted_info.get('mobile_number', ' ')
    email_address = extracted_info.get('email_address', ' ')
    website_url = extracted_info.get('website_url', ' ')
    area = extracted_info.get('area', ' ')
    city = extracted_info.get('city', ' ')
    state = extracted_info.get('state', ' ')
    pin_code = extracted_info.get('pin_code', ' ')

    # Insert the data into the database table
    sql = '''
        INSERT INTO business_cards (company_name, card_holder_name, designation, mobile_number, 
                                    email_address, website_url, area, city, state, pin_code, image)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    '''
    values = (company_name, card_holder_name, designation, mobile_number, email_address, website_url,
              area, city, state, pin_code, image_bytes)

    cursor.execute(sql, values)
    connection.commit()
    connection.close()


# ... (rest of the code)


if __name__ == "__main__":
    main()
